In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()

In [ ]:
data["Symptoms"] = 0

In [ ]:
records = data.shape[0]
for i in range(records):
    values = data.iloc[i].values
    values = values.tolist()
    # print(values)
    if 0 in values:
        data["Symptoms"][i] = values[1:values.index(0)]
    else:
        data["Symptoms"][i] = values[1:]
    if data["Disease"][i] == "Paralysis (brain hemorrhage)":
        # data["Symptoms"][i] = data["Symptoms"][i][:-1]
        if "headache" in data["Symptoms"][i]:
            data["Symptoms"][i].remove("headache")


In [ ]:
data[data["Disease"] == "Paralysis (brain hemorrhage)"]

In [ ]:
column_values = data[['Symptom_1', 'Symptom_2', 'Symptom_3', 'Symptom_4',
       'Symptom_5', 'Symptom_6', 'Symptom_7', 'Symptom_8', 'Symptom_9',
       'Symptom_10', 'Symptom_11', 'Symptom_12', 'Symptom_13', 'Symptom_14',
       'Symptom_15', 'Symptom_16', 'Symptom_17']].values.ravel()
symps = pd.unique(column_values)
symps = symps.tolist()
symps = [x.strip() for x in symps if str(x) != 'nan']
import pickle
pickle.dump(symps, open('symptoms.pkl', 'wb'))
symps

In [ ]:
symptoms = pd.DataFrame(columns=symps, index=data.index)
symptoms.head()

In [ ]:
symptoms["Symptoms"] = data["Symptoms"]
symptoms.head()

In [ ]:
for i in symps:
    symptoms[i] = symptoms.apply(lambda x:1 if i.strip() in x.Symptoms else 0, axis=1)

In [ ]:
symptoms["Disease"] = data["Disease"]
symptoms = symptoms.drop("Symptoms", axis=1)
symptoms

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(symptoms, test_size=0.2, random_state=42)
X_train = train.drop("Disease", axis=1)
y_train = train["Disease"]
X_test = test.drop("Disease", axis=1)
y_test = test["Disease"]

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
model = HistGradientBoostingClassifier(random_state=42, max_iter=1000, max_leaf_nodes=1000, learning_rate=0.1, l2_regularization=0.05 )
model.fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X_train, y_train, cv=5).mean()

In [ ]:
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
import pickle
pickle.dump(model, open('model.pkl','wb'))